In [1]:
# FIXME: remove this cd
import os
import sys
from pathlib import Path

# add path to domblar development code
sys.path.insert(0, str(Path(os.getcwd()).parent))

/Users/gexahedron/devel/domblar


In [5]:
import ipywidgets as widgets
from ipywidgets import GridspecLayout
from IPython.display import display

from copy import deepcopy
import threading
import time

from domblar.domblar import Domblar
from domblar.edo import get_freq


class Tracker():
    def __init__(self):
        self.running = False
        self.thread_is_started = False
        # FIXME
        self.thread2_is_started = False

        # FIXME: well, there's a lot to fix here
        self.clear()
        self.d = None
        self.dur = 0.25
        self.synth_idx = None
        self.voice_amps = []
        # FIXME: this is just a separate fixme for this atrocity
        self.sleep_time = 4 * self.dur
        self.rep = 1
        self.loop = True

    def clear(self):
        self.voices = None
        self.edo = None
        self.sus = None
        self.loop_from = 0
        self.loop_idx = 0

        self.next_voices = None
        self.next_edo = None
        self.next_sus = None
        self.next_dur = None
        self.next_loop_from = None
        self.set_synth_query = None

    def eternal_run(self):
        while True:
            self.single_run()

    def set_synth_monitor(self):
        while True:
            if self.set_synth_query is not None:
                # FIXME: this appoarch is very hiccup-y
                reps, whole_dur, synth_idx,\
                    instr1, instr2, shift, param_count = self.set_synth_query
                self.set_synth_query = None

                dur = whole_dur / reps
                for prop in range(reps):
                    self.d.set_synth(synth_idx, instr1, instr2,
                                     1.0 - prop / reps,
                                     shift, param_count)
                    time.sleep(dur)
            else:
                time.sleep(self.dur)

    def modulate_synth(self,
                       reps, whole_dur, synth_idx,
                       instrument1, instrument2,
                       shift=0, param_count=None):
        self.set_synth_query = (reps, whole_dur, synth_idx,
                                instrument1, instrument2,
                                shift, param_count)

    def play_voices(self):
        # FIXME: this method is too big and messy
        if self.synth_idx is None:
            self.synth_idx = list(range(len(self.voices)))
        if type(self.rep) is list:
            assert len(self.synth_idx) == len(self.rep)
        else:
            self.rep = [self.rep] * len(self.synth_idx)
        last_reps = [0] * len(self.synth_idx)

        scale = list(range(self.edo))

        last_event_idx = [0] * len(self.voices)
        if self.loop_idx > 0:
            last_event_idx = [self.loop_from] * len(self.voices)
        last_event_time = [0] * len(self.voices)
        final_sleep = 0
        start_time = time.time()
        self.latency = 0  # 0.02  # FIXME: move this somewhere else
        beats_ahead = 2  # FIXME: also move this somewhere else
        total_beat_count = 0
        for event in self.voices[0]:
            total_beat_count += event[1]

        progressed = True
        waiting = False
        while self.running and (progressed or waiting):
            progressed = False
            waiting = False
            for voice_idx in range(len(self.voices)):
                if not self.running:
                    break
                if last_event_idx[voice_idx] >= len(self.voices[voice_idx]):
                    continue
                waiting = True
                cur_beat = (time.time() - start_time) / self.dur
                self.progress.value = cur_beat / total_beat_count
                if last_event_time[voice_idx] > cur_beat + beats_ahead:
                    continue

                note, note_beat_count = self.voices[voice_idx][last_event_idx[voice_idx]]
                freq = get_freq(note, scale, self.edo)
                send_note_dur = self.dur
                if self.sus:
                    send_note_dur = self.sus
                send_note_dur *= note_beat_count
                amp = 1.0
                # FIXME:
                # if self.voice_amps:
                    # amp *= self.voice_amps[voice_idx]
                amp *= self.voice_amps[voice_idx].value

                cur_beat = (time.time() - start_time) / self.dur
                if last_event_time[voice_idx] > cur_beat + beats_ahead:
                    continue
                # FIXME: is it a right formula though?
                # i mean, the thing with latency
                timetag = start_time + last_event_time[voice_idx] * self.dur + self.latency
                final_sleep = max(final_sleep, timetag + note_beat_count * self.dur)

                self.d.client.send_note(
                    self.synth_idx[voice_idx] + last_reps[voice_idx],
                    freq=freq, dur=send_note_dur, amp=amp,
                    timetag=timetag, channel=0)  # TODO: for MPE with channels use channel=note_idx
                last_reps[voice_idx] = (last_reps[voice_idx] + 1) % self.rep[voice_idx]

                last_event_idx[voice_idx] += 1
                last_event_time[voice_idx] += note_beat_count
                progressed = True

            if waiting and not progressed:
                # sleep 1 beat
                time.sleep(self.dur)

        if self.running:
            time.sleep(final_sleep - time.time())

    def single_run(self):
        play = self.loop
        if self.next_voices is not None:
            play = True
            self.loop_idx = 0
            self.voices = deepcopy(self.next_voices)
            self.edo = self.next_edo
            self.dur = self.next_dur
            self.sus = self.next_sus
            self.loop_from = self.next_loop_from
            # FIXME
            self.sleep_time = 4 * self.dur

            self.next_voices = None
        if self.running and self.voices and play:
            self.play_voices()
            self.loop_idx += 1
        else:
            time.sleep(self.sleep_time)

    def print_state(self):
        if self.running:
            print("I'm alive")
        else:
            print("Sleeping...")

    def stop(self):
        self.running = False

    def setup_dashboard(self):
        self.voice_amps = []
        for idx in range(self.synth_count):
            layout = widgets.Layout(width='75%')
            # if idx == 0:
                # layout = widgets.Layout(width='95%')
            self.voice_amps.append(
                widgets.FloatSlider(
                    min=0,
                    max=2.0,
                    step=0.05,
                    description=f'Vol{idx}',  # TODO/FIXME: maybe it should be idx+1?
                    value=1.0,
                    orientation='vertical',
                    layout=layout,
                    readout_format='.1f',
                )
            )
        self.progress = widgets.FloatProgress(
            value=0.0,
            min=0,
            max=1.0,
            step=0.01,
            description='Loop %',
            bar_style='info',
            orientation='horizontal',
            layout=widgets.Layout(width='80%')
        )

    def show_dashboard(self):
        grid = GridspecLayout(9, len(self.voice_amps), width='50%')
        grid[0, :] = self.progress
        for voice_amp_idx, voice_amp in enumerate(self.voice_amps):
            grid[1:, voice_amp_idx] = voice_amp
        display(grid)

    def start(self, synth_count, context):
        # print('Starting domblar...')
        self.synth_count = synth_count
        self.d = Domblar(
            synth_count=self.synth_count,
            context=context,
        )
        self.clear()
        self.running = True
        if not self.thread_is_started:
            self.thread_is_started = True
            thread = threading.Thread(target=self.eternal_run)
            thread.start()
        # FIXME
        if not self.thread2_is_started:
            self.thread2_is_started = True
            thread2 = threading.Thread(target=self.set_synth_monitor)
            thread2.start()

        self.setup_dashboard()
        # print('Done!')
        self.show_dashboard()

    # FIXME: rename
    def queue(self, voices, edo, dur, sus=None, once=False, loop_from=0):
        self.running = True

        if self.next_voices:
            print("Can't queue, please wait...")
            return False
        self.next_voices = deepcopy(voices)
        self.next_edo = edo
        self.next_sus = sus
        self.next_dur = dur
        self.next_loop_from = loop_from
        self.loop = not once
        print('Queued!')
        return True


In [6]:
t = Tracker()

In [7]:
synth_count = 4

t.start(
    synth_count=synth_count,
    context='dexed',
)

GridspecLayout(children=(FloatProgress(value=0.0, bar_style='info', description='Loop %', layout=Layout(grid_a…

In [8]:
import random
from copy import copy, deepcopy
from collections import defaultdict

from domblar.edo import note_in_scale
from domblar.music_math import ratio_to_floats, ratio_to_cents, OCTAVE_IN_CENTS, C4_FREQ
from domblar.transformations import transpose


# FIXME: what to do with defaults here? should i remove them?
def cycle(a, dup_start=False, dup_end=False):
    end = -2
    if dup_end:
        end = -2
    if dup_start:
        return a + a[end::-1]
    else:
        return a + a[end:0:-1]


# NOTE/FIXME: start is 1-indexed, which is incosistent with previous code
# FIXME: "notes" is a bad naming
def sequence(start, ops,
              notes=None, final_deg=None,  # TODO/FIXME: replace with fn to check the goal conditions
              scale=None, edo=None,
              rewrite=dict(),  # FIXME: this thing needs notation
              modulate_to=dict(),  # FIXME: this thing needs same notation
              modulate_from=dict(),  # FIXME: this thing needs same (?) notation
              cutoff=1000,  # FIXME: arbitrary number
):
    # FIXME: maybe not needed assertions
    assert scale is not None
    assert edo is not None

    # assertions
    if (rewrite is not None) or (modulate_to is not None):
        assert (scale is not None) and (edo is not None)
    # assert that exactly one of these is not None
    assert (notes is not None) != (final_deg is not None)

    cur_scale = copy(scale)
    cur_deg = start
    result = []
    idx = 0
    while True:
        # check that we have achieved the goal
        if notes is not None and len(result) >= notes:
            break
        elif final_deg is not None and result[-1][0] == final_deg:
            break

        # FIXME: in theory we may never achieve the goal
        # so we need some emergency stop
        # what is the best strategy here?
        assert len(result) < cutoff

        # FIXME/TODO: what if we have same cur_deg
        # both in rewrite and in modulate_to or
        # both in rewrite and in modulate_from?

        # analyze cur_deg and cur_scale
        if (cur_deg in rewrite) or (cur_deg in modulate_to) or (cur_deg in modulate_from):
            # rewrite[degree] = (a, b)
            # where b is degree of cur_scale, and it becomes a root of next_scale
            # a is degree of next_scale
            # e. g., in heptatonic (7-note) scale
            # a/b ~= .../IV, if we treat cur_scale as V of next_scale
            if (cur_deg in modulate_from):
                assert cur_deg not in rewrite
                new_pair = modulate_from[cur_deg]
                new_root = note_in_scale(cur_scale, new_pair[1] - 1, edo) % edo
                new_scale = transpose(cur_scale, new_root - cur_scale[0])
                result.append((new_pair[0], new_scale, edo))
            if (cur_deg in rewrite) or (cur_deg in modulate_to):
                if cur_deg in modulate_to:
                    assert cur_deg not in rewrite
                    result.append((cur_deg, cur_scale, edo))
                    to_pair = modulate_to[cur_deg]
                else:
                    assert cur_deg not in modulate_to
                    to_pair = rewrite[cur_deg]
                new_root = note_in_scale(cur_scale, to_pair[1] - 1, edo) % edo
                cur_scale = transpose(cur_scale, new_root - cur_scale[0])
                cur_deg = to_pair[0]

        result.append((cur_deg, cur_scale, edo))

        # recalc cur_deg, and idx for next ops
        cur_deg = cur_deg + ops[idx]
        while cur_deg <= 0:
            cur_deg += len(scale)
        while cur_deg > len(scale):
            cur_deg -= len(scale)
        # FIXME: it's better to convert ops to a lazy infinite stream
        idx = (idx + 1) % len(ops)
    if notes is not None:
        result = result[:notes]

    return result


def build_scale_from_steps(steps, edo=None, start=0):
    if type(steps) is str:
        steps = [int(x) for x in steps]
    assert type(steps) is list
    scale = [start]
    for step in steps[:-1]:
        scale.append(scale[-1] + step)
    if edo is not None:
        assert scale[-1] + steps[-1] == scale[0] + edo
    return scale


def cents_to_steps(cents, edo, use_round=True):
    float_steps = cents / OCTAVE_IN_CENTS * edo
    if use_round:
        return round(float_steps)
    return int(float_steps)


def ratio_to_steps(m, n, edo, use_round=True):
    return cents_to_steps(ratio_to_cents(m, n), edo, use_round=use_round)


def acoustic_scale(edo, use_round=True):
    # 7 note scale, kind-of approximating harmonics 7:8:9:10:11:12:13
    # NOTE/maybe FIXME: for 12edo the usual acoustic scale uses int instead of round
    # and there are actually some reasons for this,
    # explained in papers by Dmitri Tymoczko or Marek Žabka
    # but if we use round - we still get acoustic scale in 12edo,
    # just with a different "root"
    # for other edo scales this is not true
    scale = []
    for i in range(7, 14):
        scale.append(ratio_to_steps(i, 7, edo, use_round=use_round))
    return scale


def parse_voicing(voicing):
    # TODO/FIXME: create nice syntax for voicings
    ops = dict()
    # FIXME: find a better name
    chunks = voicing.strip().split(' ')
    for chunk in chunks:
        octave = 0
        octave -= chunk.count('_')
        chunk = chunk.replace('_', '')
        octave += chunk.count('^')
        chunk = chunk.replace('^', '')
        ops[int(chunk)] = octave
    return ops


# FIXME: it's very similar to parse_voicing
# ideally i think it should be a single function for both
def parse_gesture(gesture):
    # NOTE: i assume here that gesture output is melody
    # TODO: add support for chords (won't work for Dexed; but for other MPE synths it's good)
    result = []
    if type(gesture) is str:
        gesture = gesture.strip().split(' ')

    assert type(gesture) is list

    for note_info in gesture:
        rep = 1
        if type(note_info) is str:
            if 'x' in note_info:
                note_info = note_info.split('x')
                assert len(note_info) == 2
                rep = int(note_info[-1])
                note_info = note_info[0]
            dur = 1
            if ':' in note_info:
                note_info = note_info.split(':')
                assert len(note_info) == 2
                dur = int(note_info[-1])
                note_info = note_info[0]
            # FIXME: code duplication
            octave = 0
            octave -= note_info.count('_')
            note_info = note_info.replace('_', '')
            octave += note_info.count('^')
            note_info = note_info.replace('^', '')
            note_idx = note_info
            # FIXME: explicit dot
            if note_idx != '.':
                note_idx = int(note_idx)
        # FIXME: code here should be mostly deprecated
        elif type(note_info) is dict:
            pitch_info = list(note_info.keys())[0]
            dur = list(note_info.values())[0]
            if type(pitch_info) is tuple:
                note_idx, octave = pitch_info
            else:
                note_idx = pitch_info
                octave = 0
        else:
            pitch_info = note_info
            dur = 1
            # FIXME: code duplication
            if type(pitch_info) is tuple:
                note_idx, octave = pitch_info
            else:
                note_idx = pitch_info
                octave = 0

        result.extend([(note_idx, octave, dur)] * rep)

    return result


def apply_voicing(chord, voicing, octave):
    if voicing is None:
        return chord
    voicing_ops = parse_voicing(voicing)
    # FIXME: copy chord
    for voice_idx in voicing_ops:
        chord[voice_idx] += octave * voicing_ops[voice_idx]
    return chord


def build_tonic_steps(scale, voices_count, quality):
    # TODO: add support for octatonic scales
    # FIXME: in reality there should be some analysis of the scale
    # that corresponds to the quality
    # say, that we understand, that 2 steps really produce tertian quality
    # TODO: add support for higher limits (7-, 11-, 13-)
    # TODO: add support polytonality
    assert voices_count in [3, 4]
    steps = None
    if len(scale) == 7:
        if quality == 'tertian':
            steps = [2] * (voices_count - 1)
        elif quality == 'quartal':
            steps = [3] * (voices_count - 1)
        elif quality == 'add6':
            # FIXME: this is a bit ugly
            assert voices_count == 4
            steps = [2, 2, 1]
        else:
            assert quality == 'add2'
            assert voices_count == 4
            # FIXME: it's add9, really
            # also known as mu chord
            steps = [2, 2, 4]
    else:
        assert len(scale) == 10
        # TODO: add more support for decatonic scales
        # for 7-limit tetrads, the decatonic pattern is
        # X _ _ X _ _ X _ X _ X
        quality == 'tertian'
        steps = [3, 3, 2]
        steps = steps[:voices_count]
    return steps


# FIXME: rename voices to voices_count
# FIXME: hide defaults for quality and voices
def chord_in_scale(scale, degree, voices=3, quality='tertian'):
    # FIXME: this approach won't work in non-fifth based edos
    # and scales with length not equal to 7 or 10
    steps = build_tonic_steps(scale, voices, quality)

    assert 0 < degree <= len(scale)
    root = degree - 1

    chord = []
    sum_step = 0
    for i in range(voices):
        chord.append(root + sum_step)
        if i < voices - 1:
            sum_step += steps[i]
    return chord


def chord_in_edo(scale, edo, degree, voices=3, quality='tertian', voicing=None):
    chord = chord_in_scale(scale, degree=degree, voices=voices, quality=quality)
    if voices == 3:
        chord.append(chord[0] + len(scale))
    # FIXME: hacky default voicing, remove it
    if voicing is None:
        voicing = '0_'
    chord = apply_voicing(chord, voicing, octave=len(scale))
    # TODO: add option to sort notes, so that bass is note 0
    # NOTE/maybe FIXME: we switch here from scale to full edo, maybe there's a better approach
    return [note_in_scale(scale, n, edo) for n in chord]


# FIXME: inconsistent naming
# "in_edo" in "chord_in_edo" means something different
# than "in_edo" in "cadence_in_edo"
def cadence_in_edo(chord, new_chord_shape, edo, new_root=None, jump=False):
    # FIXME: should be a function/Chord() method
    new_pc_set = set()
    new_pc_set = frozenset([n % edo for n in new_chord_shape])
    if new_root is None:
        new_root = new_chord_shape[0] % edo

    # FIXME: right now we keep only 1 candidate for each possible pc set
    # but in the future we should keep a bit more promising candidates
    possible_new_sub_chords = dict()

    # FIXME: quartal chords don't have roots
    # TODO: for each voice we should respect voice range restrictions

    # FIXME: we also implicitly assume here that bass in voice 0
    # FIXME: maybe move this variable to optional parameters
    base_is_root = jump
    first_non_jump_note = 0
    # NOTE: this tolerance is in edo steps, not in scale steps
    # smooth_leading_tolerance_in_steps = round(edo / 3)  # FIXME: heuristic
    smooth_leading_tolerance_in_steps = round(edo * 5 / 12)  # other probable heuristic
    if base_is_root:
        first_non_jump_note = 1
        # TODO: inefficient
        for diff in range(smooth_leading_tolerance_in_steps + 1):
            # FIXME: we implicitly assume here that
            # we can have only 1 candidate within the tolerance
            # although with tolerance < edo/2 this should be always true
            signs = [-1, 1]
            if diff == 0:
                signs = [-1]
            for sign in signs:
                note = chord[0] + diff * sign
                pc = note % edo
                if pc == new_root:
                    assert (not possible_new_sub_chords)
                    # FIXME: ad hoc data structure
                    # tuple (chord, edo-step penalty)
                    possible_new_sub_chords[frozenset([pc])] = ([note], 0)
            if possible_new_sub_chords:
                break
        if not possible_new_sub_chords:
            candidates = []
            for sign in [-1, 1]:  # in current implementation the order here doesn't matter
                # TODO: inefficient, should be O(1)
                for diff in range(smooth_leading_tolerance_in_steps + 1, edo):
                    note = chord[0] + diff * sign
                    pc = note % edo
                    if pc == new_root:
                        candidates.append((note, pc))
                        break
            assert len(candidates) == 2
            note, pc = random.choice(candidates)
            possible_new_sub_chords[frozenset([pc])] = ([note], 0)
    else:
        possible_new_sub_chords[frozenset()] = ([], 0)

    # now find some voice-leading for "non-bass" notes
    for i in range(first_non_jump_note, len(chord)):
        possible_new_bigger_sub_chords = dict()
        found = False  # NOTE: actually it's not needed, we will always find something
        # but i will keep it for now because the code is complicated
        for diff in range(smooth_leading_tolerance_in_steps + 1):  # FIXME: probably inefficient for large edos
            signs = random.choice([[-1, 1], [1, -1]])  # FIXME: hack, to alleviate the problem with equal penalty
            if diff == 0:
                signs = [1]
            for sign in signs:
                note = chord[i] + diff * sign
                # FIXME: should be a function/Chord() method
                pc = note % edo
                if pc in new_pc_set:
                    found = True
                    penalty = diff
                    # FIXME: confusing naming
                    for prev_pc_set in possible_new_sub_chords:
                        prev_chord, prev_penalty = possible_new_sub_chords[prev_pc_set]
                        bigger_pc_set = frozenset(list(prev_pc_set) + [pc])
                        bigger_chord = copy(prev_chord) + [note]
                        bigger_penalty = penalty + prev_penalty
                        prev_bigger_penalty = None
                        if bigger_pc_set in possible_new_bigger_sub_chords:
                            prev_bigger_penalty = possible_new_bigger_sub_chords[bigger_pc_set][1]
                        if (prev_bigger_penalty is None) or (prev_bigger_penalty > bigger_penalty):
                            possible_new_bigger_sub_chords[bigger_pc_set] = (bigger_chord, bigger_penalty)
                        # FIXME: we could have an ambiguity here,
                        # if penalties are the same
        assert found
        possible_new_sub_chords = deepcopy(possible_new_bigger_sub_chords)

    # TODO: choose best chord
    if new_pc_set in possible_new_sub_chords:
        return possible_new_sub_chords[new_pc_set][0]
    new_chord = None
    best_pc_set_size = 0
    best_penalty = None
    for pc_set in possible_new_sub_chords:
        if len(pc_set) >= best_pc_set_size:
            candidate_chord, candidate_penalty = possible_new_sub_chords[pc_set]
            if (len(pc_set) > best_pc_set_size) or candidate_penalty >= best_penalty:
                # FIXME: we could have an ambiguity here,
                # if penalties are the same
                new_chord = candidate_chord
                best_penalty = candidate_penalty
                best_pc_set_size = len(pc_set)
    assert new_chord is not None

    # FIXME: with the current approach we can have some "problems",
    # e. g.:
    # - merging of 2 voices into the same note (should be easy to fix)
    # - voice crossings (should be easy to fix)
    # - not getting the full chord quality/modality (to fix this we would need to split notes)
    # although sometimes this is actually a desired behavior
    return new_chord


def cadence(chord, scale, edo, degree,
            voices=3, quality='tertian', jump=False):
    new_chord_shape = chord_in_scale(scale, degree=degree, voices=voices, quality=quality)
    # TODO: we switch here from scale to full edo, maybe there's a better approach
    new_chord_shape = [note_in_scale(scale, n, edo) for n in new_chord_shape]
    new_root = note_in_scale(scale, degree - 1, edo) % edo
    return cadence_in_edo(chord, new_chord_shape, edo, new_root=new_root, jump=jump)



# FIXME: hide default value for voices_default
# FIXME: code duplicate
def progress(degrees, voices_default=3,
             voicing=None, start_from=None,  # FIXME: these 2 parameters are conflicting
             quality=None,
             always_jump_on=[],
):
    default_quality = quality
    chords = []
    for info, cur_scale, edo in degrees:
        # FIXME: this is ugly, need a parser
        if type(info) is tuple:
            degree, voices = info
        else:
            degree = info
            voices = voices_default

        jump = False
        quality = default_quality
        if quality is None:
            quality = 'tertian'

        # parse degree and other properties
        if type(degree) is str:
            # FIXME: this is ugly, need a parser
            if 'q' in degree:
                quality = 'quartal'
                degree = degree.replace('q', '')
            elif 'add6' in degree:
                quality = 'add6'
                degree = degree.replace('add6', '')
            elif 'add2' in degree:
                quality = 'add2'
                degree = degree.replace('add2', '')
            if 'j' in degree:
                jump = True
                degree = degree.replace('j', '')
            if 'v4' in degree:
                voices = 4
                degree = degree.replace('v4', '')
            if 'v3' in degree:
                voices = 3
                degree = degree.replace('v3', '')
            degree = int(degree)

        if degree in always_jump_on:
            jump = True

        if not chords:
            if start_from is None:
                chords.append(chord_in_edo(scale=cur_scale, edo=edo,
                                           degree=degree, voices=voices,
                                           quality=quality, voicing=voicing))
            else:
                chords.append(cadence(start_from, scale=cur_scale, edo=edo,
                                      degree=degree, voices=voices,
                                      jump=jump, quality=quality))
        else:
            chords.append(cadence(chords[-1], scale=cur_scale, edo=edo,
                                  degree=degree, voices=voices,
                                  jump=jump, quality=quality))
    return chords


def chords_to_voices(chords, gestures=None, edo=None, durs=None):
    if gestures is not None:
        assert edo is not None
        gestures = [parse_gesture(gesture) for gesture in gestures]
        result = [[] for _ in range(len(gestures))]
    else:
        result = [[] for _ in range(len(chords[0]))]

    if durs is not None:
        assert len(durs) == len(chords)
    else:
        durs = [1] * len(chords)

    for chord_idx, chord in enumerate(chords):
        if gestures is not None:
            for voice_idx, gesture in enumerate(gestures):
                motif = []
                for note_idx, octave, dur in gesture:
                    if type(note_idx) is int:
                        note = chord[note_idx] + octave * edo
                    else:
                        note = note_idx
                    motif.append((note, dur * durs[chord_idx]))
                result[voice_idx].extend(motif)
        else:
            for voice_idx in range(len(chord)):
                result[voice_idx].append((chord[voice_idx], durs[chord_idx]))
    return result


# FIXME: unused, but helpful
def l_system_stats(s):
    counts = defaultdict(int)
    max_runs = defaultdict(int)
    cur_run = 0
    prev_char = ''
    for c in s:
        counts[c] += 1
        if c == prev_char:
            cur_run += 1
        else:
            if prev_char:
                max_runs[prev_char] = max(max_runs[prev_char], cur_run)
            cur_run = 1
        prev_char = c
    max_runs[prev_char] = max(max_runs[prev_char], cur_run)
    return {'counts': counts.items(), 'max_runs': max_runs.items()}


# FIXME: rename
def run_l_system(rules, start, iterations):
    cur = start
    for c in list(rules.keys()):
        if type(rules[c]) is not list:
            rules[c] = [rules[c]]
    for _ in range(iterations):
        c_counter = defaultdict(int)
        rewritten = ''
        for c in cur:
            rewritten += rules[c][c_counter[c] % len(rules[c])]
            c_counter[c] += 1
        # FIXME
        # print(f'{i + 1}: ', rewritten.startswith(cur), l_system_stats(rewritten))
        cur = rewritten
    return cur


# FIXME: hide defaults for voices and quality
# FIXME: usually default for voices is 3
def tonnetz(scale, edo, degree=1, mj=0, mn=0, voices=4, quality='tertian', voicing=None):
    # min3 = ratio_to_floats(6, 5)
    # maj3 = ratio_to_floats(5, 4)

    # FIXME: this is for 11edo
    min3 = ratio_to_floats(11, 9)
    maj3 = ratio_to_floats(9, 7)

    shift = round(edo * maj3) * mj + round(edo * min3) * mn
    return chord_in_edo(transpose(scale, shift), edo, degree,
                        voices=voices, voicing=voicing, quality=quality)


# FIXME: write out assumptions about scales somewhere
# that 0 corresponds to C4
# and make it configurable
def freq_to_edo(freq, edo):
    return round(edo * ratio_to_floats(freq, C4_FREQ))


def recalc_chord(chord, prev_edo, new_edo):
    new_chord = []
    for n in chord:
        freq = get_freq(n, list(range(prev_edo)), prev_edo)
        new_chord.append(freq_to_edo(freq, new_edo))
    return new_chord

def add_scale_and_edo(progression, scale, edo):
    result = []
    for degree in progression:
        result.append((degree, scale, edo))
    return result


In [16]:
last_chord = None
last_edo = None
# TODO: last scale

def set_synths():
    t.d.set_synth(0, 'marimba20')
    t.d.set_synth(1, 'tub_bells')
    # t.d.set_synth(1, 'celeste')
    t.d.set_synth(2, 'celeste')
    t.d.set_synth(3, 'piano3')
set_synths()

In [28]:
# part A
# in the spirit of Sergei Zagny - Sonata

edo = 22
# Porcupine bright harmonic 11th mode - Porcupine[7] 6|0 b7
scale = build_scale_from_steps('4333324', edo, -4)

if last_chord is not None:
    last_chord = recalc_chord(last_chord, last_edo, edo)

# progression = ['1j']
# progression = ['4', '1j']
progression = ['2', '6', '5', '1j']
# progression = ['3', '4', '7', '1j']
# progression += ['5', '3', '4', '1j']
# progression = ['1j'] * 2
progression = add_scale_and_edo(progression, scale, edo)

# progression = cycle(
#     # sequence(4, [3, -1, -1],
#     # sequence(4, [-1, 5],
#     sequence(4, [-3, 2],
#                 scale=scale, edo=edo,
#               rewrite={1: (4, 4)},
#             #   modulate_from={
#             #            6: (5, 6),
#             #            2: (5, 2),
#             #            3: (5, 3),
#             #            4: (5, 4),
#             #            5: (5, 5),
#             #   },
#             #   modulate_to={1: (4, 4)},
#               notes=5,
# ))

chords = progress(progression,
    voicing='0_ 1_ 2_',
    voices_default=4,
    start_from=last_chord,
    quality='quartal',
    # quality='tertian',
    # quality='add6',
)


# print(progression)
# print(chords)


from domblar.transformations import split

gestures = [
    '0:4x2',
    # '.:2 0:2 .:2 1:2',
    # '0:2 .:1 0:2 .:1 0:2',
    # '0:2 2^:1 0:2 0^:1 0:2',

    # '3__:2 .:4 2__:2',
    '3_:8',
    # '3__:2 .:2 2__:2 .:2',
    # '3__:2 .:2 2_:1 3__:3',

    *split(['1','2','1','3'] * 2),
]

durs = [1] * (len(chords) - 1) + [3]

voices_with_dur = chords_to_voices(chords,
                                   gestures=gestures, edo=edo,
                                #    durs=durs
                                   )

dur = 0.15/1.2
if t.queue(
    voices=voices_with_dur,
    edo=edo,
    dur=dur,
    sus=dur * 0.95,
    # once=True,
):
    last_chord = chords[-1]
    last_edo = edo

Queued!


In [33]:
t.stop()

In [32]:
t.modulate_synth(200, 8, 1,
                 'harpsich1', 'tub_bells',
                 shift=30, param_count=40
                 )

In [90]:
last_chord = None

def set_synths():
    for i in range(4):
        if i == 0:
            t.d.set_synth(i, 'big_grand')
        else:
            t.d.set_synth(i, 'epiano1')
set_synths()

In [95]:
# part B
# NRT Tonnetz progressions
# PLR cycle
# L-systems for rhythm
# also in the spirit of Sergei Zagny - Sonata

rules = {
    '1': ['12', '112'],
    '2': ['221', '21']
}
l_rhythm = run_l_system(rules, '2', 9)

# Porcupine bright harmonic 11th mode - Porcupine[7] 6|0 b7
edo, steps = 15, '3222213'

steps = [int(x) for x in steps]
scale = build_scale_from_steps(steps, edo)

once = False
use_gestures = False
# use_gestures = True
# final_round = False
final_round = True

gestures = None
if use_gestures:
    gestures = [
        '0 . 3 .',
        '. 1 . 0',
        '1 . 2 .',
        '. 2 . 3'
    ]


def build_chord_loop(scale, edo, degree=1):
    voicing = '0_ 2_'
    chord_loop = []
    chord_loop.append(tonnetz(scale, edo, mn=-1, mj=0, degree=degree, voicing=voicing,
                              quality='add6'))
    chord_loop.append(tonnetz(scale, edo, degree=degree, voicing=voicing,
                              quality='add6'))
    chord_loop.append(tonnetz(scale, edo, mn=0, mj=1, degree=degree, voicing=voicing,
                              quality='add2'))
    # chord_loop.append(tonnetz(scale, edo, mn=0, mj=2, degree=degree, voicing=voicing,
    #                           quality='add2'))
    # chord_loop.append(tonnetz(scale, edo, mn=-1, mj=2, degree=degree, voicing=voicing,))
    chord_loop = cycle(chord_loop)
    return chord_loop

chord_loop = build_chord_loop(scale, edo)

from domblar.transformations import shift

# build chords
chords = [chord_loop[-1]]
if last_chord:
    chords = [last_chord]

# loop_count = 1
loop_count = 4
inblock_reps = 1
block_reps = 1

# loop_count = 3
# inblock_reps = 3
# block_reps = 3

for _ in range(loop_count):
    for _ in range(block_reps):
        for rep in range(inblock_reps):
            for chord in chord_loop:
                chords.append(cadence_in_edo(chords[-1], chord, edo))
        steps = shift(steps, 1)
        steps = [int(x) for x in steps]
        scale = build_scale_from_steps(steps, edo)
        chord_loop = build_chord_loop(scale, edo)

muls = [1]
if last_chord is not None:
    muls = []
r_idx = 0

muls_version = 2  # using l-rhythms
# muls_version = 1

if muls_version == 1:
    for _ in range(loop_count):
        for rep in range(inblock_reps):
            for chord in chord_loop:
                if (r_idx % (len(chord_loop) - 1) != 0):
                    muls.append(1)
                else:
                    muls.append(2)
                r_idx += 1
else:
    for _ in range(loop_count):
        sub_muls = []
        for rep in range(inblock_reps):
            for chord in chord_loop:
                sub_muls.append(int(l_rhythm[r_idx]))
                r_idx += 1
        muls.extend(sub_muls * block_reps)

if final_round:
    chords = chords[:1]
    for chord in chord_loop[:2]:
        chords.append(cadence_in_edo(chords[-1], chord, edo))
    muls = [1, 15, 16]
    once = True
    use_gesture = False


# some post-processing
chords = chords[:len(muls)]
loop_from = 1
if last_chord is not None:
    loop_from = 0
    chords = chords[1:]
    muls = muls[1:]


voices_with_dur = chords_to_voices(chords, durs=muls,
                                   gestures=gestures, edo=edo
)

dur = 0.25
if use_gestures:
    dur = 0.25/3

if t.queue(
    voices=voices_with_dur,
    edo=edo,
    dur=dur,
    sus=dur * 0.7,
    loop_from=loop_from,
    once=once,
):
    last_chord = chords[-1]


Queued!


In [17]:
t.stop()

In [96]:
last_chord = None
last_edo = None

def set_synths():
    t.d.set_synth(0, 'celeste')
    t.d.set_synth(1, 'epiano1')
    t.d.set_synth(2, 'celeste')
    t.d.set_synth(3, '*vibration')
set_synths()


In [102]:
# part C==A'
# decatonic scales (tertian) harmony
# 12edo, 15edo, 17edo, 22edo

# once = False
once = True
use_gestures = False
# use_gestures = True

# edo, steps = 12, '1121111211'  # Four-semitone tritone scale/Messiaen's 7th "mode"
# edo, steps = 15, '2121212112'  # Blackwood decatonic
# edo, steps = 17, '2122122212'  # neutrominant
edo, steps = 22, '3133131331'  # astrology[10] mode
# edo, steps = 22, '2232222322'  # pajara[10] symmetric decatonic

# choose random mode
steps = [int(x) for x in steps]
steps = shift(steps, random.randint(0, 9))
scale = build_scale_from_steps(steps, edo=edo, start=-edo)

# generate random sequence
x = random.randint(1, 10)
y = -x + random.randint(1, 9)
z = random.randint(2, 10)
seq = [x, y]
notes = 12
dup_start = False
z = 5; seq = [1, -2, -1, 1]; notes = 31; dup_start = True

progression = cycle(sequence(z, seq,
                              scale=scale, edo=edo,
                              notes=notes,
                              rewrite={1: (5, 6)},
), dup_start=dup_start) + [('1j', scale, edo)]


muls_for_3rd_part = []
stop_dur = 6
last_dur = 1
for idx, deg in enumerate(progression):
    if deg in [1, '1j']:
        muls_for_3rd_part.append(stop_dur)
        last_dur = stop_dur
        # stop_dur += 2
    else:
        muls_for_3rd_part.append(last_dur)
        last_dur = max(last_dur // 2, 1)

muls_for_3rd_part[-1] = stop_dur * 2

if last_chord is not None and not (-edo * 3 < min(last_chord) < 0):
    last_chord = None
if last_chord is not None:
    last_chord = recalc_chord(last_chord, last_edo, edo)

chords = progress(
    progression,
    voicing='1^',
    start_from=last_chord,
    voices_default=4,
    always_jump_on=[1],
)
print(progression)

gestures = None
if use_gestures:
    gestures = [
        '0:2',
        '1_ 2',
        '2_ 1',
        '3:2',
    ]

dur = 0.15
dur = 0.15/2
# dur = 0.15/3
if use_gestures:
    dur = dur/2

voices_with_dur = chords_to_voices(chords,
                                   gestures=gestures, edo=edo,
                                   durs=muls_for_3rd_part)

if t.queue(
    voices=voices_with_dur,
    edo=edo,
    dur=dur,
    sus=dur * 0.95,
    loop_from=1,
    once=once,
):
    last_chord = chords[-1]
    last_edo = edo

[(5, [-22, -19, -16, -15, -12, -11, -8, -5, -4, -1], 22), (6, [-22, -19, -16, -15, -12, -11, -8, -5, -4, -1], 22), (4, [-22, -19, -16, -15, -12, -11, -8, -5, -4, -1], 22), (3, [-22, -19, -16, -15, -12, -11, -8, -5, -4, -1], 22), (4, [-22, -19, -16, -15, -12, -11, -8, -5, -4, -1], 22), (5, [-22, -19, -16, -15, -12, -11, -8, -5, -4, -1], 22), (3, [-22, -19, -16, -15, -12, -11, -8, -5, -4, -1], 22), (2, [-22, -19, -16, -15, -12, -11, -8, -5, -4, -1], 22), (3, [-22, -19, -16, -15, -12, -11, -8, -5, -4, -1], 22), (4, [-22, -19, -16, -15, -12, -11, -8, -5, -4, -1], 22), (2, [-22, -19, -16, -15, -12, -11, -8, -5, -4, -1], 22), (5, [11, 14, 17, 18, 21, 22, 25, 28, 29, 32], 22), (6, [11, 14, 17, 18, 21, 22, 25, 28, 29, 32], 22), (7, [11, 14, 17, 18, 21, 22, 25, 28, 29, 32], 22), (5, [11, 14, 17, 18, 21, 22, 25, 28, 29, 32], 22), (4, [11, 14, 17, 18, 21, 22, 25, 28, 29, 32], 22), (5, [11, 14, 17, 18, 21, 22, 25, 28, 29, 32], 22), (6, [11, 14, 17, 18, 21, 22, 25, 28, 29, 32], 22), (4, [11, 14, 17

In [167]:
t.stop()

In [19]:
finetuning = 0
synth_idx = 1

try:
    finetuning
except Exception as e:
    t.d.open_editor(synth_idx)
    finetuning = 1
else:
    if finetuning == 0:
        t.d.open_editor(synth_idx)
    elif finetuning == 1:
        # transfer instrument to all synths
        t.d.save_preset(synth_idx)
        import time
        time.sleep(0.5)
        for i in range(synth_count):
            t.d.load_preset(i)
    else:
        t.d.print_params(synth_idx)

In [12]:
# cleanup
t.d.stop_server()

In [18]:
last_chord = None
last_edo = None
# TODO: last scale

def set_synths():
    for i in range(4):
        t.d.set_synth(i, 'epiano1')
    # t.d.set_synth(0, 'marimba20')
set_synths()

In [82]:
# edo, steps = 12, '2212221'
edo, steps = 22, '4333324'  # Porcupine bright harmonic 11th mode - Porcupine[7] 6|0 b7
scale = build_scale_from_steps(steps, edo=edo, start=0)

last_chord = None

progression = sequence(1, [1],
    scale=scale, edo=edo,
    modulate_from={
        2: (5, 2),
        3: (5, 3),
        4: (5, 4),
        5: (5, 5),
        6: (5, 6),
    },
    notes=13)

for i in range(len(progression)):
    if progression[i][1] == scale:
        progression[i] = (f'{progression[i][0]}j', progression[i][1], progression[i][2])
    else:
        progression[i] = (f'{progression[i][0]}add2', progression[i][1], progression[i][2])

print(progression)

chords = progress(progression,
    # voicing='0_ 2_',
    voicing='1^',
    voices_default=4,
    start_from=last_chord,
    # quality='quartal',
    # quality='tertian',
    # quality='add2',
    # quality='add6',
)

for chord in chords:
    print([chord[0], chord[2], chord[3], chord[1]])

from domblar.transformations import split

gestures = [
    # '0_:4x2',
    # '.:2 0:2 .:2 1:2',
    # '0_:2 .:1 0:2 .:1 0_:2',
    '0_:2 2:1 0:2 1:1 0_:2',

    # '3_:2 .:4 2_:2',
    '3__:2 .:2 2__:2 .:2',
    # '3__:2 .:2 2_:1 3__:3',

    *split(['1',2,'1','3'] * 2),
]

voices_with_dur = chords_to_voices(chords,
                                #    gestures=gestures, edo=edo
)

dur = 0.5/4
dur = 0.5
if t.queue(
    voices=voices_with_dur,
    edo=edo,
    dur=dur,
    sus=dur * 0.95,
    once=True,
):
    last_chord = chords[-1]
    last_edo = edo

[('1j', [0, 4, 7, 10, 13, 16, 18], 22), ('5add6', [4, 8, 11, 14, 17, 20, 22], 22), ('2j', [0, 4, 7, 10, 13, 16, 18], 22), ('5add6', [7, 11, 14, 17, 20, 23, 25], 22), ('3j', [0, 4, 7, 10, 13, 16, 18], 22), ('5add6', [10, 14, 17, 20, 23, 26, 28], 22), ('4j', [0, 4, 7, 10, 13, 16, 18], 22), ('5add6', [13, 17, 20, 23, 26, 29, 31], 22), ('5j', [0, 4, 7, 10, 13, 16, 18], 22), ('5add6', [16, 20, 23, 26, 29, 32, 34], 22), ('6j', [0, 4, 7, 10, 13, 16, 18], 22), ('7j', [0, 4, 7, 10, 13, 16, 18], 22), ('1j', [0, 4, 7, 10, 13, 16, 18], 22)]
[0, 13, 18, 29]
[0, 11, 17, 30]
[4, 10, 16, 22]
[3, 11, 14, 20]
[7, 4, 13, 18]
[6, 1, 14, 17]
[10, 0, 7, 16]
[9, -2, 4, 17]
[13, -4, 4, 10]
[12, -2, 1, 7]
[16, -9, 0, 7]
[18, -6, 4, 10]
[22, -4, 7, 13]
Queued!


In [101]:
t.stop()